In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install unidecode

!pip install torchtext==0.2.3

! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz

    100% |████████████████████████████████| 235kB 7.2MB/s 
    100% |████████████████████████████████| 51kB 3.5MB/s 
  Running setup.py bdist_wheel for torchtext ... - \ done
  Stored in directory: /root/.cache/pip/wheels/42/a6/f4/b267328bde6bb680094a0c173e8e5627ccc99543abded97204
Successfully built torchtext
--2018-10-16 04:10:27--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 52.20.136.189, 52.55.184.84, 34.200.202.18, ...
Connecting to piazza.com (piazza.com)|52.20.136.189|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2018-10-16 04:10:27--  https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import unidecode
import string
import random
import re
 
import pdb

import torch
import torch.nn as nn
from torch.autograd import Variable
import time
import numpy as np
import matplotlib.pyplot as plt

 
all_characters = string.printable
n_characters = len(all_characters)
 
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 2579888


In [0]:
chunk_len = 200
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]
 
print(random_chunk())


# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
      tensor[c] = all_characters.index(string[c])
  return Variable(tensor)
 
print(char_tensor('abcDEF'))
 
def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

ards. It ran cunningly, taking a line that seemed chosen so as to keep 
as much hidden as possible from the view, both of the hill-tops above and of 
the flats to the west. It dived into dells, and hug
tensor([10, 11, 12, 39, 40, 41])


In [0]:
#PART 3
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers=1):
    super(GRU, self).__init__()
    
    self.z_sig = nn.Sigmoid()
    self.r_sig = nn.Sigmoid()
    self.tanh = nn.Tanh()
    
    self.lin1 = nn.Linear(hidden_size, 100)
    self.lin2 = nn.Linear(hidden_size, 100)
   
 
  def forward(self, input, hidden):
    z = self.z_sig(torch.cat((hidden, input), 2))
    print(z.size())
    r = self.r_sig(torch.cat((hidden, input), 2))
    print(r.size())
    h_r = self.lin1(hidden * r)
    h_hat = self.tanh(torch.cat((h_r, self.lin2(input)), 1))
    print(h_hat.size())
    return (1 - z) * hidden + z * h_hat
    
    

In [0]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    
    # encode using embedding layer
    # set up GRU passing in number of layers parameter (nn.GRU)
    self.GRU = GRU(input_size=input_size, hidden_size=hidden_size, num_layers=n_layers)
    self.out = nn.Linear(hidden_size, output_size)
    self.embedding = nn.Embedding(input_size, hidden_size)
    
    # decode output

  def forward(self, input, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU
    # return output and hidden.
    input = self.embedding(input).view(1, 1, -1) 
    output, hidden = self.GRU(input, hidden)
    return  self.out(output[0]), hidden

  def init_hidden(self):
    return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
    

In [0]:
def train(inp, target):
  
  hidden = decoder.init_hidden()
  loss = 0
  decoder_optimizer.zero_grad()
  
  
  for c in range(chunk_len):
    output, hidden = decoder(inp[c], hidden)# run the forward pass of your rnn with proper input
    loss += criterion(output, target[c].view(1))

  loss.backward()
  decoder_optimizer.step()

  return loss.item() / chunk_len

In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):

  prime_input = char_tensor(prime_str)
  
  hidden = decoder.init_hidden()
  predicted = ""

# Use priming string to "build up" hidden state
  for p in range(len(prime_str) - 1):
    _, hidden = decoder(prime_input[p], hidden)
  inp = prime_input[-1]

  for p in range(predict_len):
    output, hidden = decoder(inp, hidden) #run your RNN/decoder forward on the input

    # Sample from the network as a multinomial distribution
    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]

    #top_i convert to char append

    ## get character from your list of all characters, add it to your output str sequence, set input
    predicted += all_characters[top_i]
    inp = char_tensor(all_characters[top_i])

  return predicted

In [ ]:
n_epochs = 2000
print_every = 40
plot_every = 15
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
      print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
      print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
      all_losses.append((epoch, loss_avg / plot_every))
      loss_avg = 0

In [0]:
loss_x, loss_y = zip(*all_losses)
plt.plot(loss_x, loss_y, label='Losses')
plt.legend()
plt.title("LOSS")
plt.show()
